# Using the pluto MutualFundsIndia data-set

In [1]:
library(dbplyr)
library(dplyr)
library(odbc)
library(plutoR)
options("scipen"=999)
source("config.R")

#initialize
mfi <- MutualFundsIndia()


Attaching package: ‘dplyr’

The following objects are masked from ‘package:dbplyr’:

    ident, sql

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



### who are the biggest 5 mutual fund managers?

In [2]:
lastAumDate <- (mfi$AumFundwise() %>%
    summarize(MAX_TS = max(PERIOD)) %>%
    collect())$MAX_TS[1]

mfi$AumFundwise() %>%
    filter(PERIOD == lastAumDate) %>%
    select(FUND) %>%
    mutate(AUM = AVG_AUM_WO_FOFD + AVG_AUM_FOFD) %>%
    arrange(desc(AUM)) %>%
    print(n=5)


Warning message:
“Missing values are always removed in SQL.
Use `MAX(x, na.rm = TRUE)` to silence this warning
This warning is displayed only once per session.”

# Source:     lazy query [?? x 2]
# Database:   Microsoft SQL Server 13.00.4224[ro1@NORWAY/StockViz]
# Ordered by: desc(AUM)
  FUND                                    AUM
  <chr>                                 <dbl>
1 HDFC Mutual Fund                  36276279.
2 ICICI Prudential Mutual Fund      33876820.
3 SBI Mutual Fund                   30784117.
4 Aditya Birla Sun Life Mutual Fund 25418198.
5 Reliance Mutual Fund              22327193.
# … with more rows


### what are the largest 10 mid-cap funds by AUM?

In [3]:
# Meta to filter for the 'Mid-Cap' category
# AumSchemewise to sort funds by AUM - largest first
# NavTimeSeries to get the start and end dates for which NAVs are available

lastMetaDate <- (mfi$Meta() %>%
    summarize(MAX_TS = max(AS_OF)) %>%
    collect())$MAX_TS[1]

lastSwDate <- (mfi$AumSchemewise() %>%
    summarize(MAX_TS = max(PERIOD)) %>%
    collect())$MAX_TS[1]

print(lastMetaDate)
print(lastSwDate)

[1] "2019-06-22"
[1] "2019-06-01"


In [4]:
mfAum <- mfi$Meta() %>% 
    inner_join(mfi$NavTimeSeries(), by="SCHEME_CODE") %>%
    rename(SN = SCHEME_NAME) %>% #rename to avoid conflict in the group_by
    inner_join(mfi$AumSchemewise(), by="SCHEME_CODE") %>%
    group_by(SCHEME_CODE, SCHEME_NAME, AVG_AUM_WO_FOFD) %>%
    filter(PERIOD == lastSwDate & AS_OF == lastMetaDate & CATEGORY == 'Mid-Cap') %>%
    summarize(start_dt = min(TIME_STAMP), end_dt = max(TIME_STAMP)) %>%
    arrange(desc(AVG_AUM_WO_FOFD)) 

mfAum %>% print(n=10)

Warning message:
“Missing values are always removed in SQL.
Use `MIN(x, na.rm = TRUE)` to silence this warning
This warning is displayed only once per session.”

# Source:     lazy query [?? x 5]
# Database:   Microsoft SQL Server 13.00.4224[ro1@NORWAY/StockViz]
# Groups:     SCHEME_CODE, SCHEME_NAME
# Ordered by: desc(AVG_AUM_WO_FOFD)
   SCHEME_CODE SCHEME_NAME                 AVG_AUM_WO_FOFD start_dt   end_dt    
         <int> <chr>                                 <dbl> <date>     <date>    
 1      104481 DSP  Midcap Fund - Regular…         486411. 2006-11-16 2019-07-01
 2      100473 Franklin India Prima Fund-…         475444. 2006-04-03 2019-07-01
 3      101539 Sundaram Mid Cap Fund -Gro…         467403. 2006-04-03 2019-07-01
 4      112496 L&amp;T Mid Cap Fund-Regul…         345492. 2010-02-16 2019-07-01
 5      118533 Franklin India Prima Fund …          98004. 2013-01-01 2019-07-01
 6      119807 L&amp;T Mid Cap Fund-Direc…          93321. 2013-01-02 2019-07-01
 7      119071 DSP  Midcap Fund - Direct …          82532. 2013-01-02 2019-07-01
 8      119581 Sundaram Mid Cap Fund- Dir…          46320. 2013-01-02 2019-07-01
 9      105503

###  get the NAV time-series

In [5]:
scode <- as.integer(mfAum[[1]][1])
print(scode)

ERROR: Error in eval(expr, envir, enclos): (list) object cannot be coerced to type 'integer'


This notebook was created using [pluto](http://pluto.studio). Learn more [here](https://github.com/shyams80/pluto)